In [30]:
import os
# Import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import CohereEmbeddings
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
index_name = "langchain-test-index-3"

# index_name = "cohere-test-embeddings"
existing_indexes = [
    index["name"] for index in pc.list_indexes()
]
index_exists = index_name in existing_indexes

# Create an Index
if not index_exists:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

embeddings_model = CohereEmbeddings(
    model="embed-english-light-v3.0",
    cohere_api_key=os.environ.get("COHERE_API_KEY"),
)

vectorstore_a = PineconeVectorStore(
    embedding=embeddings_model,
    index_name=index_name
)

In [31]:
vectorstore_a.add_texts(["USC is the best school in the world", 
                         "USC is known for being awesome because it is very warm during the summer",
                        "USC has the best college of Letter's of Arts and Sciences called Dornsife Viterbi",
                         "USC has the best dance school, Kauffman, better than any other dance school in the world (Juliard)",
                         "USC is known for being bad because it has many hard linguistics and CS classes",
                         "USC is bad because Complex Analysis is too Complex, and Accounting is not accounted for"]
                        )

['e0f2a0ce-f797-4869-869d-f08171aa2f3e',
 '1b4ac9dc-5351-4571-8380-7f813e61d62d',
 'aa2e4750-3e4a-455c-b963-7c6e2259bba4',
 '6d6e72fb-431d-4478-af78-ae1792eb963e',
 '076517fe-f4b2-41cf-95fc-8221d4433231',
 '565eac3e-ca60-4cdf-999b-12cf540f93de']

In [32]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.llms import Cohere

llm = Cohere(model="command")

retriever = vectorstore_a.as_retriever()
prompt_str = """
Question: {question}

Please answer the question above using only the context provided.
Context: {context}

Answer:
"""

prompt =  PromptTemplate.from_template(prompt_str)

retrieve_context_and_get_quality = RunnableParallel({"question": RunnablePassthrough(), "context": retriever})

chain = retrieve_context_and_get_quality | prompt | llm  | StrOutputParser()

In [33]:
out = chain.invoke("Why is USC bad?")
print(out)

 Some reasons that USC is considered bad is that the school has very difficult linguistics and CS classes on top of the fact that the Complex Analysis class is considered too complex and the Accounting department is not well regarded.


In [34]:
out = chain.invoke("Why is USC good?")
print(out)

 USC is known for its awesome and warm summer as well as for being one of the best schools in the world, however it is also known for its difficult linguistics and CS classes. Complex analysis is also considered too complex, and accounting is not accounted for, therefore it's important to consider the positive and negative attributes of USC when evaluating the school. 


In [35]:
# Cleanup
# pc.delete_index(name=index_name)